In [1]:
using JuMP, Cbc, Ipopt

In [2]:
A = [6 4; 1 2; -1 1; 0 1]
b = [24; 6; 1; 2]
c = [5; 4]

2-element Array{Int64,1}:
 5
 4

## Solving the primal and dual problems
We formulate and solve the two problems to obtain their optimal solutions

In [3]:
primalmodel = Model(Cbc.Optimizer)
@variable(primalmodel, x[1:2] >= 0)
@objective(primalmodel, Max, sum(c.*x))
@constraint(primalmodel, [m in 1:4], sum(A[m,:].*x) <= b[m])
optimize!(primalmodel)
value.(primalmodel[:x])

Presolve 3 (-1) rows, 2 (0) columns and 6 (-1) elements
0  Obj -0 Dual inf 8.999998 (2)
2  Obj 21
Optimal - objective value 21
After Postsolve, objective 21, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 21 - 2 iterations time 0.002, Presolve 0.00


2-element Array{Float64,1}:
 2.9999999999999996
 1.5000000000000002

In [4]:
dualmodel = Model(Cbc.Optimizer)
@variable(dualmodel, p[1:4] >= 0)
@objective(dualmodel, Min, sum(b.*p))
@constraint(dualmodel, [n in 1:2], sum(A[:,n].*p) >= c[n])
optimize!(dualmodel)
value.(dualmodel[:p])

Presolve 2 (0) rows, 4 (0) columns and 7 (0) elements
0  Obj 0 Primal inf 1.8333331 (2)
4  Obj 21
Optimal - objective value 21
Optimal objective 21 - 4 iterations time 0.002


4-element Array{Float64,1}:
 0.7499999999999998
 0.5000000000000004
 0.0
 0.0

## Complementary slackness
We verify the optimality of our solutions using complementary slackness

In [5]:
x_opt = value.(primalmodel[:x])
p_opt = value.(dualmodel[:p]) 
lhs_primal = A*x_opt
lhs_dual = A'*p_opt

println("Primal feasibility:")
for m in 1:4
    println("$(round(lhs_primal[m],digits=2)) <= $(b[m])")
end

println("\nDual feasibility:")
for n in 1:2
    println("$(round(lhs_dual[n],digits=2)) >= $(c[n])")
end

println("\nComplementary slackness:")
for m in 1:4
    println("$(round((lhs_primal[m] - b[m])*p_opt[m],digits=2)) = 0")
end
for n in 1:2
    println("$(round((lhs_dual[n] - c[n])*x_opt[n],digits=2)) = 0")
end

Primal feasibility:
23.999999999999996 <= 24
6.0 <= 6
-1.4999999999999993 <= 1
1.5000000000000002 <= 2

Dual feasibility:
4.999999999999998 <= 5
4.0 <= 4

Complementary slackness:
-2.664535259100375e-15 = 0
0.0 = 0
-0.0 = 0
-0.0 = 0
-5.3290705182007506e-15 = 0
0.0 = 0


## Bonus: Solving the primal and dual problems at the same time 
### A nonlinear complementarity model

In [6]:
complementarity = Model(Ipopt.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Ipopt

In [7]:
@variable(complementarity, x[1:2] >= 0)
@variable(complementarity, p[1:4] >= 0)

4-element Array{VariableRef,1}:
 p[1]
 p[2]
 p[3]
 p[4]

In [8]:
@expression(complementarity, pr_con[m in 1:4], b[m] - sum(A[m,:].*x))
@expression(complementarity, du_con[n in 1:2], sum(A[:,n].*p) - c[n])

@constraint(complementarity, [m in 1:4], pr_con[m] >= 0)
@constraint(complementarity, [n in 1:2], du_con[n] >= 0)

@constraint(complementarity, [m in 1:4], pr_con[m]*p[m] <= 0) # "Should" be == instead of <=, but this is more stable.
@constraint(complementarity, [n in 1:2], du_con[n]*x[n] <= 0) # If two variables are both nonnegative, their product is also nonnegative. 
                                                              # Combining that with a nonpositivity constraint, the effect is the same as a equality constraint

2-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarQuadraticFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1}:
 6 p[1]*x[1] + p[2]*x[1] - p[3]*x[1] - 5 x[1] <= 0.0
 4 p[1]*x[2] + 2 p[2]*x[2] + p[3]*x[2] + p[4]*x[2] - 4 x[2] <= 0.0

In [9]:
optimize!(complementarity)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       48
Number of nonzeros in Lagrangian Hessian.............:       14

Total number of variables............................:        6
                     variables with only lower bounds:        6
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equal

In [10]:
value.(complementarity[:x])

2-element Array{Float64,1}:
 3.0000000183333335
 1.5000000008333325

In [11]:
value.(complementarity[:p])

4-element Array{Float64,1}:
 0.7499999954864254
 0.5000000007857839
 0.0
 5.0000368467492094e-9

In [12]:
println("Primal objective value: $(sum(c.*value.(complementarity[:x])))")
println("Dual objective value:   $(sum(b.*value.(complementarity[:p])))")

Primal objective value: 21.000000094999997
Dual objective value:   20.999999906388986
